## Question 2 & 3
#### Data filter, process, and load into Neo4j Knowledge Graph

In [1]:
import json

data = []
with open('data/train.json', 'r', encoding='utf-8') as file:
    for line in file:
        data.append(json.loads(line))


In [13]:
def save_json(data_to_save,file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)


In [3]:
import pandas as pd


def extract_relation_types(data_src):
    merged = []
    for d in data_src:
        for p in d['passages']:
            merged.extend(p['exhaustivelyAnnotatedProperties'])

    factes_df = pd.DataFrame(merged)
    factes_df = factes_df.drop_duplicates().sort_values('propertyId')
    factes_df.to_csv('data/relations.csv', index=False)
    return factes_df


extract_relation_types(data)


,propertyId,propertyName,propertyDescription
1,1,SUBSIDIARY_OF,Describes the relationship between a parent co...
3,10,NATIONALITY,Describes he country of a person's citizenship...
24,11,PLACE_OF_RESIDENCE,Describes the relationship between a person an...
33,12,PLACE_OF_BIRTH,Describes the relationship between a person an...
2,14,DATE_OF_DEATH,Describes the date of death of a person.
35,15,DATE_OF_BIRTH,Describes the date of birth of a person.
11,2,FOUNDED_BY,Describes the relationship between an organiza...
12,25,SPOUSE,Describes the relationship between a person an...
19,3,EMPLOYEE_OR_MEMBER_OF,Describes the previous/current employer of a p...
39,34,CHILD_OF,Describes the relationship between a person an...


sample document

In [11]:
data[2]


{'fold': 1,
 'source': 'Diffbot Knowledge Graph',
 'documentId': '29726',
 'documentText': 'Morphine\n\nMorphine was an American alternative rock group formed by Mark Sandman, Dana Colley, and Jerome Deupree in Cambridge, Massachusetts in 1989. After five successful albums and extensive touring, they disbanded after lead vocalist Sandman died of a heart attack onstage in Palestrina, Italy, on July 3, 1999. Founding members have reformed into the band Vapors of Morphine, maintaining much of the original style and sound.',
 'passages': [{'passageId': '29726:151:315',
   'exhaustivelyAnnotatedProperties': [{'propertyId': '14',
     'propertyName': 'DATE_OF_DEATH',
     'propertyDescription': 'Describes the date of death of a person.'}],
   'passageStart': 151,
   'passageEnd': 315,
   'passageText': 'After five successful albums and extensive touring, they disbanded after lead vocalist Sandman died of a heart attack onstage in Palestrina, Italy, on July 3, 1999.',
   'facts': [{'factId': 

In [6]:
import pandas as pd

relations_df = pd.read_csv('data/relations.csv')
relations_id_map = {k[0]: k for k in relations_df.values}
relations_name_map = {k[1]: k for k in relations_df.values}


relations given in this assignment

In [7]:
relations_scope = [relations_name_map[r.split(" ")[0]][0] for r in """DATE_OF_BIRTH (PER–DATE)
PLACE_OF_RESIDENCE (PER–LOC) 
PLACE_OF_BIRTH (PER–LOC)
NATIONALITY (PER–LOC)
EMPLOYEE_OR_MEMBER_OF (PER–ORG) 
EDUCATED_AT (PER–ORG) """.split('\n')]
# relations_scope.sort()
relations_scope


[15, 11, 12, 10, 3, 9]

FIltering data

In [14]:
data_with_facts = []
all_facts = []
print(f'Original document count = {len(data)}')
for d in data:
    passages_with_facts = []
    for p in d['passages']:
        facts = []
        for f in p['facts']:
            if int(f['propertyId']) in relations_scope and str(f['subjectText']).lower() not in ['i', 'my', 'he', 'she', 'his', 'her', 'who', 'myself'] and str(f['objectText']).lower() not in ['where', 'them', 'they', 'when', 'its']:
                facts.append(f)
                all_facts.append(f)
        p['facts'] = facts
        # print(len(p['facts']))
        if len(p['facts']) > 0:
            passages_with_facts.append(p)
    d['passages'] = passages_with_facts
    if len(d['passages']) > 0:
        data_with_facts.append(d)
print(f'Document with given relations count = {len(data_with_facts)}')
print(f'Total fact count = {len(all_facts)}')
save_json(data_with_facts,'data/filtered_train.json')
save_json(all_facts,'data/filtered_all_facts.json')


Original document count = 3977
Document with given relations count = 1115
Total fact count = 2462


trying to see data

In [11]:
for d in data_with_facts[1000:1010]:
    print(f'{d["documentId"]}:')
    # print(f'{d["documentText"]}:')
    for p in d['passages']:
        print(f'\t{p["passageId"]}:')
        for f in p['facts']:
            print(f'\t\t{f["humanReadable"]}:')


27831:
	27831:22:122:
		<María Ángela Holguín Cuéllar> <DATE_OF_BIRTH> <13 November 1963>:
24347:
	24347:24:158:
		<Bob Lynch> <NATIONALITY> <Australian>:
29692:
	29692:19:128:
		<Alberto Camenzind> <NATIONALITY> <Swiss>:
21733:
	21733:17:200:
		<Francesco Rocca> <NATIONALITY> <Italian>:
5222:
	5222:14:156:
		<Nikolaus "Klaus" Barbie> <EMPLOYEE_OR_MEMBER_OF> <Gestapo>:
	5222:157:290:
		<Butcher of Lyon> <EMPLOYEE_OR_MEMBER_OF> <Gestapo>:
17743:
	17743:229:419:
		<Markopoulos> <PLACE_OF_RESIDENCE> <Europe>:
		<they> <PLACE_OF_RESIDENCE> <United States>:
		<they> <PLACE_OF_RESIDENCE> <Europe>:
		<Gregory Markopoulos> <PLACE_OF_RESIDENCE> <Europe>:
		<Gregory Markopoulos> <PLACE_OF_RESIDENCE> <United States>:
		<Markopoulos> <PLACE_OF_RESIDENCE> <United States>:
7957:
	7957:15:100:
		<Russell J. Eddie> <DATE_OF_BIRTH> <June 9, 1938>:
	7957:101:135:
		<Eddie> <PLACE_OF_BIRTH> <Nebraska>:
		<Eddie> <PLACE_OF_BIRTH> <Wayne>:
8857:
	8857:17:150:
		<Walid Ali Osman> <PLACE_OF_BIRTH> <Libya>:
	

### Generate query and load data into Neo4J Knowledge Graph

In [12]:
from neo4j import GraphDatabase

# Connect to Neo4j
uri = "bolt://localhost:7687"
username = "etl_user"
password = "etl_user123"
driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session(database="knowledgenet")

# Iterate through the folds
for fact in all_facts:
    try:
        subjectText = str(fact['subjectText']).replace("'", "\\'")
        objectText = str(fact['objectText']).replace("'", "\\'")
        relation = relations_id_map[int(fact['propertyId'])][1]

        query = f"MERGE (p:Person {{name: '{subjectText}'}}) \
                                ON CREATE SET p.name = '{subjectText}'"

        print(query)
        session.run(query)

        if fact['propertyId'] in ['10', '11', '12']:
            # Loc
            query = f"MERGE (p:Location {{name: '{objectText}'}}) \
                                ON CREATE SET p.name = '{objectText}'"
            session.run(query)
            # Create the  relationship
            query = f"MATCH (d:Person {{name: '{subjectText}'}}), (p:Location {{name: '{objectText}'}}) MERGE (d)-[:{relation}]->(p)"

            print(query)
            session.run(query)

        if fact['propertyId'] in ['3', '9']:
            # Org
            query = f"MERGE (p:Organization {{name: '{objectText}'}}) \
                                ON CREATE SET p.name = '{objectText}'"
            print(query)
            session.run(query)
            # Create the  relationship
            query = f"MATCH (d:Person {{name: '{subjectText}'}}), (p:Organization {{name: '{objectText}'}}) MERGE (d)-[:{relation}]->(p)"
            print(query)
            session.run(query)

        if fact['propertyId'] in ['15']:
            # Org
            query = f"MERGE (p:Date {{name: '{objectText}'}}) \
                                ON CREATE SET p.name = '{objectText}'"
            print(query)
            session.run(query)
            # Create the  relationship
            query = f"MATCH (d:Person {{name: '{subjectText}'}}), (p:Date {{name: '{objectText}'}}) MERGE (d)-[:{relation}]->(p)"
            print(query)
            session.run(query)
    except:
        print('failed to insert')


MERGE (p:Person {name: 'بد الله بن محمد بن سعود آل ثاني'})                                 ON CREATE SET p.name = 'بد الله بن محمد بن سعود آل ثاني'
MATCH (d:Person {name: 'بد الله بن محمد بن سعود آل ثاني'}), (p:Location {name: 'Qatari'}) MERGE (d)-[:NATIONALITY]->(p)
MERGE (p:Person {name: 'Abdullah bin Mohammed bin Saud Al Thani'})                                 ON CREATE SET p.name = 'Abdullah bin Mohammed bin Saud Al Thani'
MATCH (d:Person {name: 'Abdullah bin Mohammed bin Saud Al Thani'}), (p:Location {name: 'Qatari'}) MERGE (d)-[:NATIONALITY]->(p)
MERGE (p:Person {name: 'Jim Harris'})                                 ON CREATE SET p.name = 'Jim Harris'
MATCH (d:Person {name: 'Jim Harris'}), (p:Location {name: 'American'}) MERGE (d)-[:NATIONALITY]->(p)
MERGE (p:Person {name: 'James Patrick Harris'})                                 ON CREATE SET p.name = 'James Patrick Harris'
MATCH (d:Person {name: 'James Patrick Harris'}), (p:Location {name: 'American'}) MERGE (d)-[:NATIONALITY]->